In [104]:
import pandas as pd
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import re

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error

predictors=['spingoid_backbone_dbl_bonds','fatty_acyl_dbl_bonds',\
'spingoid_backbone_carb', 'fatty_acyl_carb','mass_squared','mass_sqrt','log_mass','Mass']#'Mass',



#st.write(file_type)


clf_lass1 = linear_model.Lasso(alpha=0.1)
mod_ridge = Ridge(alpha=1.0)
mod_ridge2 = Ridge(alpha=0.9)
mod_ridge3 = Ridge(alpha=0.4)
#mod_xgb = xgb.XGBRegressor()
model_lr = LinearRegression()
model_svm = SVR(kernel = 'rbf')
mod_xgb = xgb.XGBRegressor(max_depth=2,learning_rate=0.05)
clf_lass = linear_model.Lasso(alpha=0)
clf_lass3=linear_model.Lasso(alpha=0.5)


def findx(string):

	#based on molecule structure being of the form 'Cer (d18:1/18:1)' returns key variables
	b1=string.find('(')
	d=string.find('/')
	b2=string.find(')')
	c1=string[b1+2:b1+4]
	dc1=string[d-1:d]
	c2=string[d+1:d+3]
	dc2=string[d+4:d+5]
	oh=0
	if re.search('OH',string):
		oh=1
	
	return c1,dc1,c2,dc2,oh

def recode_carbs(df):
    df['carbs']=df['Lipid ID'].apply(findx)
    for i,c in enumerate(['spingoid_backbone_carb', 'spingoid_backbone_dbl_bonds','fatty_acyl_carb', 'fatty_acyl_dbl_bonds',
                        'OH']):
        df[c]=df['carbs'].apply(lambda x:x[i])
        df[c]=pd.to_numeric(df[c],errors='coerce')
    return df

def mass_trans(df,verbose=False):

	
	#creates more variables for modelling
	df['log_mass']=df['Mass'].apply(lambda x:np.log(x))
	df['mass_squared']=df['Mass'].apply(lambda x:pow(x,2))
	df['mass_sqrt']=df['Mass'].apply(lambda x:pow(x,0.5))
	
	return df

#3 - split train and test


def recast_train_test(df,train_ratio=0.5):
    lips=list(df['Lipid ID'].sample(int(df.shape[0]*train_ratio)))
    df['Type']='Train'
    mask=df['Lipid ID'].isin(lips)
    df.loc[mask,'Type']='Test'

    return df

def train_test_try(df,preds,depvar='RT'):
    
	mask1=(df['Type']=="Train")
	X_train=df.loc[mask1,preds]
	y_train=df.loc[mask1,depvar]
	mask2=(df['Type']=="Test")
	X_test=df.loc[mask2,preds]
	y_test=df.loc[mask2,depvar]

    
	return X_train,y_train,X_test,y_test

scaler = StandardScaler()
def scale_df(X,cols):
    X[cols]=scaler.fit_transform(X[cols])
    return X

def select_data(df=df_all,study='1 - Cers 2 Study ',species='SM',exp='SM Hippocampus Data'):
    df=df[study]
    mask=(df['Species']==species)&(df['Experiment']==exp)
    df=df.loc[mask,]
    return df

def select_data_simp(df=df_all,species='SM',exp='SM Hippocampus Data'):
    mask=(df['Species']==species)&(df['Experiment']==exp)
    df=df.loc[mask,]
    return df

path="/Users/michaelallwright/Documents/data/lipid/raw/"
file=path+"summary_file_all.xlsx"
file2=path+"Sphingolipid list_with RTs.xlsx"

In [105]:
df_all2=pd.read_excel(file2,sheet_name=None)

In [106]:
df_all2.keys()

dict_keys(['Ceramide', 'Sphingomyelin '])

In [107]:
exps=['Exp 1.1 (Hippocampus Data)', 'Exp 1.2 (Temporal cortext  grey data)',
       'Exp 1.3 (Temporal cortex white)', 'Exp 1.4 (Frontal cortex grey)',
       'Exp 1.5 (Frontal cortex white)', 'Exp 1.6 (Cerebellum)',
       'Exp 2 (ApoE Study)', 'Exp 3.1 (Mouse Brain)', 'Exp 3.2 (Mouse Liver)',
       'Exp 4 (AD Mouse model)', 'Exp 5 (Lung Cancer Study)',
       'Exp 6 (Cers 1 Study)', 'Exp 7 (MSA Study)', 'Exp 8 (Insulin Study)',
       'Exp 9 (Diabetes Study)', 'Exp 10 (myelin paper)', 'Exp 11 (FTD Paper)',
       'Exp 12 (B-cells paper)', 'Exp 13 (Ovarian Cancer Paper)']

id_vars=['Lipid ID','Mass']

def transpose_data(df=df_all2,species='Sphingomyelin ',species_name='SM'):
    
       df=df[species].rename(columns={'Exact Mass':'Mass'})
       df=pd.melt(df[id_vars+exps],id_vars=id_vars)
       df.rename(columns={'variable':'Experiment','value':'RT'},inplace=True)
       mask=pd.notnull(df['RT'])
       df=df.loc[mask,]
       df['Species']=species_name

       return df
    
df_mod_sm=transpose_data(df=df_all2,species='Sphingomyelin ',species_name='SM')
df_mod_cer=transpose_data(df=df_all2,species='Ceramide',species_name='Ceramide')

df_mod=pd.concat([df_mod_sm,df_mod_cer],axis=0)

In [108]:
mask=(df_mod['Experiment']=='Exp 3.1 (Mouse Brain)')
df_mod.loc[mask,]

,Lipid ID,Mass,Experiment,RT,Species
303,SM (d18:1/12:0),646.505000,Exp 3.1 (Mouse Brain),11.35,SM
305,SM (d18:1/16:0),702.567600,Exp 3.1 (Mouse Brain),13.18,SM
306,SM (d18:1/18:0),730.598900,Exp 3.1 (Mouse Brain),13.94,SM
307,SM (d18:1/20:0),758.630200,Exp 3.1 (Mouse Brain),14.61,SM
308,SM (d18:1/22:0),786.661500,Exp 3.1 (Mouse Brain),15.2,SM
310,SM (d18:1/24:0),814.692800,Exp 3.1 (Mouse Brain),15.76,SM
311,SM (d18:1/25:0),828.708400,Exp 3.1 (Mouse Brain),9.75,SM
312,SM (d18:1/26:0),842.724100,Exp 3.1 (Mouse Brain),10.48,SM
313,SM (d18:1/16:1),700.551900,Exp 3.1 (Mouse Brain),12.62,SM
314,SM (d18:1/18:1),728.583200,Exp 3.1 (Mouse Brain),13.49,SM


In [109]:
studies_list=[]
experiments_list=[]
species_list=[]
train_sizes=[]
test_sizes=[]
r2_scores=[]
rmse_scores=[]
model_names_list=[]


clf_lass1 = linear_model.Lasso(alpha=0.1)
mod_ridge = Ridge(alpha=1.0)
mod_ridge2 = Ridge(alpha=0.9)
mod_ridge3 = Ridge(alpha=0.4)
#mod_xgb = xgb.XGBRegressor()
model_lr = LinearRegression()
model_svm = SVR(kernel = 'rbf')
mod_xgb = xgb.XGBRegressor(max_depth=2,learning_rate=0.05)
clf_lass = linear_model.Lasso(alpha=0)
clf_lass3=linear_model.Lasso(alpha=0.5)

models=[clf_lass1,mod_ridge,mod_ridge2,mod_ridge3,model_lr,model_svm,mod_xgb,clf_lass,clf_lass3]
model_names=['lass1','ridge','ridge2','ridge0.4','linear regression','svm','xgb','lass2','lass0.5']

species=['SM','Ceramide']

df=df_mod.copy()

species=list(df['Species'].unique())
experiments=list(df['Experiment'].unique())

for rat in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]:
    for spec in species:
        for exp in experiments:
            df1=select_data_simp(df,species=spec,exp=exp)
            df1.columns=[c.strip() for c in df1.columns]

            df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
            df1['Lipid ID']=df1['Lipid ID'].astype(str)
            mask=(df1['RT']>1)&pd.notnull(df1['RT'])&pd.notnull(df1['Mass'])&pd.notnull(df1['Lipid ID'])&(~df1['Lipid ID'].str.contains('!'))
            df1=df1.loc[mask,]

            df1=recode_carbs(df1)
            df1=mass_trans(df1)
            df1=recast_train_test(df1,train_ratio=rat)#0.5)
        
            X_train,y_train,X_test,y_test=train_test_try(df1,preds=predictors)

            predcols2=[c for c in X_train.columns if X_train[c].nunique()>1]
            X_train=X_train[predcols2]
            X_test=X_test[predcols2]

            train_size=len(y_train)
            test_size=len(y_test)

            for i,mod in enumerate(models):
                try:
                
                    mod =mod.fit(X_train, y_train)
                    #df1['lass_ret']=mod.predict(df1[predcols2])
                    y_pred=mod.predict(X_test)
                    r2=r2_score(y_test, y_pred)
                    print(r2)
                    rmse=mean_squared_error(y_test, y_pred)
                    print(rmse)
                except:
                    r2=np.nan
                    rmse=np.nan
            
            
                experiments_list.append(exp)
                species_list.append(spec)
                train_sizes.append(train_size)
                test_sizes.append(test_size)
                r2_scores.append(r2)
                rmse_scores.append(rmse)
                model_names_list.append(model_names[i])

/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Lipid ID']=df1['Lipid ID'].astype(str)
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective 

nan
0.06507875720093101
nan
0.00023411491079137974
nan
0.0005257551341095891
nan
0.004358264209971024
nan
0.009658214572031929
nan
0.8518432727502875
nan
0.27449282801568153
nan
0.006291166518092523
nan
0.4308642083982026
nan
0.08602506191608977
nan
0.13142306496437797
nan
0.1333749048686087
nan
0.1431757396815862
nan
0.16264134171939623
nan
4.015752226323041


/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:59: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  mod =mod.fit(X_train, y_train)
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

nan
0.3417415835704315
nan
0.05814349846664422
nan
0.93010805629816
nan
0.3258308059339592
nan
0.21776340961441548
nan
0.20972823538405466
nan
0.16684990807801642
nan
0.07092541248580644
nan
0.7975510429748852
nan
0.8718805137843809
nan
0.13018236154752558
nan
0.8018436021665635
nan
0.1837408069808004
nan
0.056697657741909636
nan
0.05295550679887804
nan
0.03417327263315713
nan
0.0009626478835699603
nan
0.22207410475887587


/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:59: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  mod =mod.fit(X_train, y_train)
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

...

/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:59: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  mod =mod.fit(X_train, y_train)
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_598

-5.291262644624868
4.042765375435945
-10.574074074074058
7.437499999999998
-1449.0845821403286
931.8243524833763
0.8973079327747141
0.09327220367601884
0.8864053047955522
0.10317474206046237
0.8832916917776052
0.10600274577511293
0.8433006806583916
0.14232541251175815
0.6441245023736251
0.32323131469437005
0.05852498104162951
0.8551142468631354
0.097903119413277
0.8193482345330381
0.37426086062532105
0.5683405742312371
0.8973257092654047
0.0932560578090095
0.5502502441393577
2.0338892560271256
0.533954368619586
2.1075835842737187
0.5326997638521935
2.1132572441783855
0.522879668663528
2.1576663535489287
0.5055432054403806
2.236066499024482
-0.51398613759159
6.84665216355487
-0.7385268963217131
7.8620858147581965


/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:59: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  mod =mod.fit(X_train, y_train)
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_598

0.7333052551057934
1.2060653045630483
0.550487938597551
2.032814337812193
-40.35036410431278
41.499458548937056
-40.40144442487203
41.550723046641785
-40.40144753586814
41.550726168855014
-40.40146309085378
41.550741779926305
-40.401475534847805
41.550754268788864
-0.33380130543007147
1.3386105101173764
-0.01225535500518582
1.0159051813910362
-0.5695544385925617
1.575213683738244
-40.1461845626494
41.29454280975698


/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Lipid ID']=df1['Lipid ID'].astype(str)
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3674820769.py:59: UserWarning: With alpha=0, this algorithm does not converge well. You are ad

In [110]:
df_sum=pd.DataFrame({'Experiment':experiments_list,'Species':species_list,'Model':model_names_list,
                     'Training data size':train_sizes,'Validation data size':test_sizes,'R2':r2_scores,'RMSE':rmse_scores})

In [111]:
df_sum.to_csv('/Users/michaelallwright/Documents/data/lipid/processed/rmse_output.csv')

In [35]:
#f=pd.read_excel(path+"summary_file_all.xlsx",sheet_name='1 - Cers 2 Study ')

df_all=pd.read_excel(file,sheet_name=None)

In [12]:
studies_list=[]
experiments_list=[]
species_list=[]
train_sizes=[]
test_sizes=[]
r2_scores=[]
rmse_scores=[]

studies=list(df_all.keys())
species=['SM','Ceramide']

for stud in studies:
    df=df_all[stud]

    species=list(df['Species'].unique())
    experiments=list(df['Experiment'].unique())

    for spec in species:
        for exp in experiments:
            df1=select_data_simp(df,species=spec,exp=exp)
            df1.columns=[c.strip() for c in df1.columns]

            df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
            df1['Lipid ID']=df1['Lipid ID'].astype(str)
            mask=(df1['RT']>1)&pd.notnull(df1['RT'])&pd.notnull(df1['Mass'])&pd.notnull(df1['Lipid ID'])&(~df1['Lipid ID'].str.contains('!'))
            df1=df1.loc[mask,]

            df1=recode_carbs(df1)
            df1=mass_trans(df1)
            df1=recast_train_test(df1)
            X_train,y_train,X_test,y_test=train_test_try(df1,preds=predictors)

            predcols2=[c for c in X_train.columns if X_train[c].nunique()>1]
            X_train=X_train[predcols2]
            X_test=X_test[predcols2]

            train_size=len(y_train)
            test_size=len(y_test)

            try:
                lass =clf_lass.fit(X_train, y_train)
                df1['lass_ret']=lass.predict(df1[predcols2])
                y_pred=lass.predict(X_test)
                r2=r2_score(y_test, y_pred)
                print(r2)
                rmse=mean_squared_error(y_test, y_pred)
                print(rmse)
            except:
                r2=np.nan
                rmse=np.nan
            
            studies_list.append(stud)
            experiments_list.append(exp)
            species_list.append(spec)
            train_sizes.append(train_size)
            test_sizes.append(test_size)
            r2_scores.append(r2)
            rmse_scores.append(rmse)

/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3273806108.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3273806108.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Lipid ID']=df1['Lipid ID'].astype(str)
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3273806108.py:41: UserWarning: With alpha=0, this algorithm does not converge well. You are ad

0.9794080950614743
0.13076592707779658
0.9847032617045762
0.08057956141983978
0.962279207584134
0.21413675153691347
0.9768911084679338
0.07220056026178032
0.9911189953355432
0.0680826739613917
0.9708696565746545
0.1260494429503177
0.9742307667813445
0.131078608850586
0.9261392328433417
0.32098430428167046
0.9798111893107666
0.11860208455544576
0.9700481701637709
0.23562937173602252
0.9674645762574716
0.2185651943853414
0.9799533037253086
0.11554247509522914
0.992912910954849
0.04898857461594104
0.9371452678274342
0.3308435650353436
0.983589506600458
0.014023587134578666
0.9946157208353414
0.004852144124466439
-0.12468732090365786
3.320532469672563
0.07447147317389036
2.820386218010375
0.9943673393639212
0.003450504298203302
nan
0.7678041012637752
0.99105066925709
0.05027282412710732
0.984560555116821
0.0975441945111562
0.9438667173211064
0.06203512118487435
nan
1.072916666655636
0.9958774597393402
0.024979691711254987
0.9695191346700175
0.24282735650992116
0.9547936755170452
0.02220207

/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.422e-03, tolerance: 2.579e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_59806/3273806108.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/v

In [14]:
df_sum=pd.DataFrame({'Study':studies_list,'Experiment':experiments_list,'Species':species_list,'Training data size':train_sizes,'Validation data size':test_sizes,'R2':r2_scores,'RMSE':rmse_scores})

In [18]:
df_sum.to_csv('/Users/michaelallwright/Documents/data/lipid/processed/training_rmse.csv')

'11- FTD ceramide data': Lipids not there
Some have unpopulated retention times


In [255]:
#loop through all studies, all experiments, SM and Ceramide
#collect R2, RMSE for each
#collect train data size and test data size

df1=select_data(df=df_all,study='13 - Ovarian Cancer paper',species='Ceramide',exp='Ovarian Cancer')
df1.columns=[c.strip() for c in df1.columns]

df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
mask=(df1['RT']>1)&pd.notnull(df1['RT'])&pd.notnull(df1['Mass'])&pd.notnull(df1['Lipid ID'])&(~df1['Lipid ID'].str.contains('!'))
df1=df1.loc[mask,]

df1=recode_carbs(df1)
df1=mass_trans(df1)
df1=recast_train_test(df1)
X_train,y_train,X_test,y_test=train_test_try(df1,preds=predictors)

predcols2=[c for c in X_train.columns if X_train[c].nunique()>1]
X_train=X_train[predcols2]
X_test=X_test[predcols2]

train_size=len(y_train)
test_size=len(y_test)

lass =clf_lass.fit(X_train, y_train)
df1['lass_ret']=lass.predict(df1[predcols2])
y_pred=lass.predict(X_test)
r2=r2_score(y_test, y_pred)
print(r2)
rmse=mean_squared_error(y_test, y_pred)
print(rmse)

0.9933235546147432
0.03481162227234252


/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/2252515445.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['RT']=pd.to_numeric(df1['RT'],errors='coerce')
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/2252515445.py:21: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lass =clf_lass.fit(X_train, y_train)
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/michaelallwright/opt/anaconda3/envs/env_full/lib/p

In [251]:
r2

0.9739434830102276

In [246]:
for c in X_train.columns:
    mask=X_train[c].astype(str).str.contains('!')
    print(c,sum(mask))

spingoid_backbone_dbl_bonds 1
fatty_acyl_dbl_bonds 0
fatty_acyl_carb 0
mass_squared 0
mass_sqrt 0
log_mass 0
Mass 0


In [248]:
df1["Lipid ID"]

0     Cer (d18:0/16:0)
2     Cer (d18:0/24:0)
3     Cer (d18:!/12:0)
5     Cer (d18:1/16:0)
7     Cer (d18:1/18:0)
8     Cer (d18:1/20:0)
9     Cer (d18:1/22:0)
10    Cer (d18:1/23:0)
11    Cer (d18:1/24:0)
14    Cer (d18:1/16:1)
15    Cer (d18:1/18:1)
17    Cer (d18:1/22:1)
18    Cer (d18:1/23:1)
19    Cer (d18:1/24:1)
25    Cer (d18:2/22:0)
30    Cer (d18:2/23:1)
Name: Lipid ID, dtype: object

In [247]:
X_train['spingoid_backbone_dbl_bonds']

3     !
5     1
9     1
10    1
11    1
15    1
17    1
18    1
Name: spingoid_backbone_dbl_bonds, dtype: object

In [215]:
r2_score(lass.predict(X_test),y_test)

0.9710081243612323

In [168]:
exps=['Hippocampus Data ', 'Temporal Cortex (Grey) Data ',
       'Temporal Cortex (White-matter) Data ',
       'Frontal Cortex (Grey) Data ', 'Frontal Cortex (White) Data ',
       'Cerebellum']

for e in exps:
    df1=select_data(df=df_all,study='1 - Cers 2 Study ',species='SM',exp=e)
    df1.columns=[c.strip() for c in df1.columns]
    df1=recode_carbs(df1)
    df1=mass_trans(df1)
    df1=recast_train_test(df1)
    X_train,y_train,X_test,y_test=train_test_try(df1,preds=predictors)

    predcols2=[c for c in X_train.columns if X_train[c].nunique()>1]
    X_train=X_train[predcols2]
    X_test=X_test[predcols2]

    lass =clf_lass.fit(X_train, y_train)
    df1['lass_ret']=lass.predict(df1[predcols2])
    r2=r2_score(lass.predict(X_test),y_test)
    print(r2)

0.9672425191773464
0.9558609927711855
0.8921259344296293
0.9784208956018469
0.9832231386500316
0.9863829958672876


/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/3695635900.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['carbs']=df['Lipid ID'].apply(findx)
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/3695635900.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c]=df['carbs'].apply(lambda x:x[i])
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/3695635900.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


['fatty_acyl_dbl_bonds',
 'fatty_acyl_carb',
 'mass_squared',
 'mass_sqrt',
 'log_mass',
 'Mass']

In [112]:
studies=['1 - Cers 2 Study ',
 '2 - ApoE-age study ',
 #'3 - Sphingadiene Paper',
 '4 - AD mouse model',
 '5 - Lung tissue',
 '6 - Cers1 study',
 '7 - MSA study',
 '8 - Insulin signalling study',
 '9 - Diabetes paper',
 '10 - myelin paper',
 '12 - B cells paper',
 '13 - Ovarian Cancer paper']

species=['SM','Ceramide']
for k in studies:

    
    df1=df_all[k]

    #display(df.head())
    
    for s in species:

        #file_out='new_retenton_time_predictions.csv'

        #truncate columns
      
        mask=df1['Species']==s
        df2=df1.loc[mask,]
        exps=list(df2['Experiment'].unique())
        #display(df.head())

        for e in exps:
           
            mask=df2['Experiment']==e
            df3=df2.loc[mask,]

            print(k,s,e,df3.shape)



1 - Cers 2 Study  SM Hippocampus Data  (18, 7)
1 - Cers 2 Study  SM Temporal Cortex (Grey) Data  (18, 7)
1 - Cers 2 Study  SM Temporal Cortex (White-matter) Data  (18, 7)
1 - Cers 2 Study  SM Frontal Cortex (Grey) Data  (18, 7)
1 - Cers 2 Study  SM Frontal Cortex (White) Data  (18, 7)
1 - Cers 2 Study  SM Cerebellum (18, 7)
1 - Cers 2 Study  Ceramide Hippocampus Data  (20, 7)
1 - Cers 2 Study  Ceramide Temporal Cortex (Grey) Data  (20, 7)
1 - Cers 2 Study  Ceramide Temporal Cortex (White-matter) Data  (20, 7)
1 - Cers 2 Study  Ceramide Frontal Cortex (Grey) Data  (20, 7)
1 - Cers 2 Study  Ceramide Frontal Cortex (White) Data  (20, 7)
1 - Cers 2 Study  Ceramide Cerebellum (20, 7)
2 - ApoE-age study  SM apoe age study (18, 7)
2 - ApoE-age study  Ceramide apoe age study (20, 7)
4 - AD mouse model SM AD Mouse model (18, 52)
4 - AD mouse model Ceramide AD Mouse model (16, 52)
5 - Lung tissue SM Lung Tissue (18, 7)
5 - Lung tissue Ceramide Lung Tissue (20, 7)
6 - Cers1 study SM Cers1 study (

In [110]:
studies=['1 - Cers 2 Study ',
 '2 - ApoE-age study ',
 #'3 - Sphingadiene Paper',
 '4 - AD mouse model',
 '5 - Lung tissue',
 '6 - Cers1 study',
 '7 - MSA study',
 '8 - Insulin signalling study',
 '9 - Diabetes paper',
 '10 - myelin paper',
 '12 - B cells paper',
 '13 - Ovarian Cancer paper']

species=['SM','Ceramide']
for k in studies:

    print(k)
    df=df_all[k]

    display(df.head())
    
    for s in species:

        #file_out='new_retenton_time_predictions.csv'

        #truncate columns
      
        mask=df['Species']==s
        df=df.loc[mask,]
        #display(df.head())
        #mask=(df['RT']==0)|(df['RT']=="ND")|pd.notnull(df['RT'])
        #df=df.loc[~mask,]
        df.columns=[c.strip() for c in df.columns]

        dfx=df.copy()
        print(dfx)
        exps=list(dfx['Experiment'].unique())

        print(exps)

        for e in exps:
            print(e)
           

            mask=dfx['Experiment']==e
            df=dfx.loc[mask,]

            display(df.head())

            
            #st.write(df)
            df.dropna(axis=1,inplace=True)
            df=mass_trans(df)

            dfk=df.copy()

            mask=~df['RT'].astype(str).str.contains("ND")&pd.notnull(df['RT'])
            df=df.loc[mask,]
            df['RT']=pd.to_numeric(df['RT'])
            mask=(df['RT']!=0)

            df['carbs']=df['Lipid ID'].apply(findx)


            for i,c in enumerate(['spingoid_backbone_carb', 'spingoid_backbone_dbl_bonds','fatty_acyl_carb', 'fatty_acyl_dbl_bonds',
                                'OH']):
                df[c]=df['carbs'].apply(lambda x:x[i])


        
            mask=(df['Type']=="Train")
            df_train=df.loc[mask,]

            preds2=[p for p in predictors if df_train[p].nunique()>1]

        
            df=scale_df(df,preds2)
            
            
            
            X_train,y_train,X_test,y_test=train_test_try(df,preds=preds2)

            # DIFFERENT POTENTIAL MODELS BELOW
            #reg =clf_reg.fit(X_train, y_train)
            #ridge=clf_ridge.fit(X_train, y_train)
            #xgb_mod_trained=xgb_mod.fit(X_train, y_train)
            
            lass =clf_lass.fit(X_train, y_train)
            
            df['lass_ret']=lass.predict(df[preds2])

            display(X_test)

            #try:

            print(r2_score(lass.predict(X_test),y_test))
            print(X_test.shape,X_train.shape)
        #except:
        #   print("fail")

1 - Cers 2 Study 


,Experiment,Species,Lipid ID,Mass,RT,Ref/Std?,Type
0,Hippocampus Data,Ceramide,Cer (d18:0/16:0),540.4,16.45,Y,Train
1,Hippocampus Data,Ceramide,Cer (d18:0/18:0),568.7,17.41,Y,Train
2,Hippocampus Data,Ceramide,Cer (d18:0/24:0),652.9,22.45,Y,Train
3,Hippocampus Data,Ceramide,Cer (d18:1/14:0),510.6,15.46,Y,Train
4,Hippocampus Data,Ceramide,Cer (d18:1/16:0),538.6,16.34,Y,Train


            Experiment Species         Lipid ID   Mass     RT Ref/Std?   Type
20   Hippocampus Data       SM  SM (d18:1/12:0)  647.5   5.19        Y  Train
21   Hippocampus Data       SM  SM (d18:1/14:0)  675.5   5.70        N   Test
22   Hippocampus Data       SM  SM (d18:1/16:0)  703.6   6.27        N   Test
23   Hippocampus Data       SM  SM (d18:1/18:0)  731.6   6.93        Y  Train
24   Hippocampus Data       SM  SM (d18:1/20:0)  759.6   7.77        N   Test
..                 ...     ...              ...    ...    ...      ...    ...
223         Cerebellum      SM  SM (d18:1/22:1)  785.6  18.22        N   Test
224         Cerebellum      SM  SM (d18:1/23:1)  799.6  18.87        N   Test
225         Cerebellum      SM  SM (d18:1/24:1)  813.7  19.56        N   Test
226         Cerebellum      SM  SM (d18:1/25:1)  827.7  20.39        N   Test
227         Cerebellum      SM  SM (d18:1/26:1)  841.7  21.27        N   Test

[108 rows x 7 columns]
['Hippocampus Data ', 'Temporal Cortex (

,Experiment,Species,Lipid ID,Mass,RT,Ref/Std?,Type
20,Hippocampus Data,SM,SM (d18:1/12:0),647.5,5.19,Y,Train
21,Hippocampus Data,SM,SM (d18:1/14:0),675.5,5.70,N,Test
22,Hippocampus Data,SM,SM (d18:1/16:0),703.6,6.27,N,Test
23,Hippocampus Data,SM,SM (d18:1/18:0),731.6,6.93,Y,Train
24,Hippocampus Data,SM,SM (d18:1/20:0),759.6,7.77,N,Test


/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/278785091.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1,inplace=True)
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/2942834857.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['log_mass']=df['Mass'].apply(lambda x:np.log(x))
/var/folders/32/kh5qwj796vs8n4y4kbm_6_lm0000gn/T/ipykernel_53829/2942834857.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

...

-8.462107648859654
(9, 5) (9, 5)
Empty DataFrame
Columns: [Experiment, Species, Lipid ID, Mass, Type, RT, 740_CB, 740_CX, 740_HC, 741_CB, 741_HC, 744_CB, 744_CX, 744_HC, 745_CB, 745_CX, 745_HC, 746_CB, 746_CX, 746_HC, 749_CB, 749_CX, 749_HC, 750_CB, 750_CX, 750_HC, 751_CB, 751_CX, 751_HC, 754_CB, 754_CX, 754_HC, 755_CB, 755_CX, 755_HC, 756_CB, 756_CX, 756_HC, 763_CB, 763_CX, 763_HC, 764_CB, 764_CX, 764_HC, log_mass, mass_squared, mass_sqrt, carbs, spingoid_backbone_carb, spingoid_backbone_dbl_bonds, fatty_acyl_carb, fatty_acyl_dbl_bonds, OH, lass_ret]
Index: []

[0 rows x 54 columns]
[]
5 - Lung tissue


,Experiment,Species,Lipid ID,Mass,RT,Ref/Std?,Type
0,Lung Tissue,Ceramide,Cer (d18:0/16:0),540.4,16.611780,Y,Train
1,Lung Tissue,Ceramide,Cer (d18:0/18:0),568.7,17.597228,Y,Type
2,Lung Tissue,Ceramide,Cer (d18:0/24:0),652.9,22.227443,Y,Train
3,Lung Tissue,Ceramide,Cer (d18:1/14:0),510.6,15.361984,Y,Type
4,Lung Tissue,Ceramide,Cer (d18:1/16:0),538.6,16.280861,Y,Train


     Experiment Species         Lipid ID   Mass         RT Ref/Std?   Type
20  Lung Tissue      SM  SM (d18:1/12:0)  647.5  13.762402        Y  Train
21  Lung Tissue      SM  SM (d18:1/14:0)  675.5  14.848405        N   Type
22  Lung Tissue      SM  SM (d18:1/16:0)  703.6  15.788452        N   Test
23  Lung Tissue      SM  SM (d18:1/18:0)  731.6  16.628817        Y   Type
24  Lung Tissue      SM  SM (d18:1/20:0)  759.6  17.766948        N   Test
25  Lung Tissue      SM  SM (d18:1/22:0)  787.6  19.009492        N   Type
26  Lung Tissue      SM  SM (d18:1/23:0)  801.6  19.751990        N   Test
27  Lung Tissue      SM  SM (d18:1/24:0)  815.7  20.431543        Y   Type
28  Lung Tissue      SM  SM (d18:1/25:0)  829.7  21.299158        N   Test
29  Lung Tissue      SM  SM (d18:1/26:0)  843.7  22.461657        N   Type
30  Lung Tissue      SM  SM (d18:1/16:1)  701.6  15.135744        N   Test
31  Lung Tissue      SM  SM (d18:1/18:1)  729.6  16.067269        N   Type
32  Lung Tissue      SM  

,Experiment,Species,Lipid ID,Mass,RT,Ref/Std?,Type
20,Lung Tissue,SM,SM (d18:1/12:0),647.5,13.762402,Y,Train
21,Lung Tissue,SM,SM (d18:1/14:0),675.5,14.848405,N,Type
22,Lung Tissue,SM,SM (d18:1/16:0),703.6,15.788452,N,Test
23,Lung Tissue,SM,SM (d18:1/18:0),731.6,16.628817,Y,Type
24,Lung Tissue,SM,SM (d18:1/20:0),759.6,17.766948,N,Test


ValueError: at least one array or dtype is required

In [103]:
exps

[]

In [60]:
X_test

,spingoid_backbone_dbl_bonds,fatty_acyl_dbl_bonds,fatty_acyl_carb,mass_squared,mass_sqrt,log_mass,Mass
9,None,None,None,None,None,None,None
11,None,None,None,None,None,None,None
12,None,None,None,None,None,None,None
14,None,None,None,None,None,None,None
15,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...
223,None,None,None,None,None,None,None
224,None,None,None,None,None,None,None
225,None,None,None,None,None,None,None
226,None,None,None,None,None,None,None


In [36]:
df[preds2+['RT']].sample(20)

,spingoid_backbone_dbl_bonds,fatty_acyl_dbl_bonds,fatty_acyl_carb,mass_squared,mass_sqrt,log_mass,Mass,RT
93,-0.699544,1.429503,-1.285645,0.094402,0.198216,0.231849,0.163943,12.86
79,1.429503,1.429503,-0.699357,-1.217741,-1.296557,-1.321223,-1.270952,12.96
105,1.429503,-0.699544,1.059505,1.361987,1.297556,1.273651,1.320317,15.39
49,1.429503,-0.699544,-0.699357,0.393444,0.480578,0.506890,0.452778,13.79
22,1.429503,-0.699544,-0.699357,-1.200921,-1.273829,-1.296278,-1.250321,13.91
8,-0.699544,-0.699544,0.473218,-0.685313,-0.629260,-0.606624,-0.649955,15.55
35,-0.699544,-0.699544,0.473218,1.050108,1.048729,1.044517,1.051081,15.20
32,-0.699544,-0.699544,-1.285645,0.115375,0.218569,0.251856,0.184574,13.18
77,1.429503,-0.699544,0.473218,-0.704797,-0.651980,-0.630366,-0.671618,15.02
91,-0.699544,-0.699544,1.352649,1.558204,1.448124,1.410510,1.485366,9.75


In [11]:
X_test

,spingoid_backbone_dbl_bonds,fatty_acyl_dbl_bonds,fatty_acyl_carb,mass_squared,mass_sqrt,log_mass,Mass


In [81]:
df

,Experiment,Species,Lipid ID,Mass,RT,Ref/Std?,Type,log_mass,mass_squared,mass_sqrt,carbs,spingoid_backbone_carb,spingoid_backbone_dbl_bonds,fatty_acyl_carb,fatty_acyl_dbl_bonds,OH,lass_ret
0,Hippocampus Data,Ceramide,Cer (d18:0/16:0),-1.495869,16.45,Y,Train,-1.594995,-1.396891,-1.545624,"(18, 0, 16, 0, 0)",18,-3.41565,-1.267542,-0.807573,0,16.558183
1,Hippocampus Data,Ceramide,Cer (d18:0/18:0),-1.203290,17.41,Y,Train,-1.236605,-1.162013,-1.221107,"(18, 0, 18, 0, 0)",18,-3.41565,-0.752391,-0.807573,0,17.910573
2,Hippocampus Data,Ceramide,Cer (d18:0/24:0),-0.332790,22.45,Y,Train,-0.267170,-0.392305,-0.300644,"(18, 0, 24, 0, 0)",18,-3.41565,0.793061,-0.807573,0,21.795262
3,Hippocampus Data,Ceramide,Cer (d18:1/14:0),-1.803956,15.46,Y,Train,-1.993264,-1.631262,-1.896666,"(18, 1, 14, 0, 0)",18,0.29277,-1.782692,-0.807573,0,14.878281
4,Hippocampus Data,Ceramide,Cer (d18:1/16:0),-1.514479,16.34,Y,Train,-1.618421,-1.411424,-1.566549,"(18, 1, 16, 0, 0)",18,0.29277,-1.267542,-0.807573,0,16.268143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Cerebellum,SM,SM (d18:1/22:1),1.039126,18.22,N,Test,1.031935,1.036146,1.036814,"(18, 1, 22, 1, 0)",18,0.29277,0.277910,1.238278,0,15.256229
224,Cerebellum,SM,SM (d18:1/23:1),1.183865,18.87,N,Test,1.155958,1.202219,1.171087,"(18, 1, 23, 1, 0)",18,0.29277,0.535486,1.238278,0,15.802869
225,Cerebellum,SM,SM (d18:1/24:1),1.329637,19.56,N,Test,1.278691,1.372442,1.305137,"(18, 1, 24, 1, 0)",18,0.29277,0.793061,1.238278,0,16.338819
226,Cerebellum,SM,SM (d18:1/25:1),1.474376,20.39,N,Test,1.398467,1.544402,1.437092,"(18, 1, 25, 1, 0)",18,0.29277,1.050636,1.238278,0,16.869571
